Progetto: **Classificazione di testi descrittivi per destinazione d'uso, inerenti ai beni sottoposti ad aste giudiziarie italiane.**

Studente: **Alessandro Monolo** | *10439147*

Relatore: Marco Brambilla

Referente aziendale: Simone Redaelli

Master: Data Science & Artificial Intelligence

Università: Politecnico di Milano

<hr style="border:1px solid black">

# Check correttezza valori Target Variable: "Destinazione d'Uso"

- **1.** Al fine di controllare la veridicità del campo (target variable) "**Destinazione d'uso**", è necessario controllare la distribuzione dei beni per singola classe di appartenenza all'interno della medesima variabile.


- **2.** Successivamente è necessario capire quali "**Categorie Catastali**" sono inerenti alle singole Destinazioni d'uso;
    - A tal proposito è necessario importare una **Transcodifica** esterna, dove per ogni Destinazione d'uso corrisponde la relativa Categoria Catastale di riferimento. Il file relativo alla transcodifica è un'export aziendale.


- **3.** Successivamente, controllare attraverso una cross-tab per la distribuzione / count dei beni, per categoria catastale.


- **4.** Elimino le categorie catastali preseti e non transcodificate inerenti ai beni presenti nel data frame;


- **5.** Infine, come secondo ed ultimo check, filtrare il data frame ottenuto, in N sub-data frame, uno per ogni destinazione d'uso, e contare i beni presenti. Per eseguire tale operazione è necessario filtrare appunto il data frame principale, selezionando per ogni sub-data frame le singole categorie catastali che compongono la destinazione d'uso di riferimento.


- **6.** Coclusioni;


- **7.** Export Data frame filtrato.

<hr style="border:1px solid black">

**Importo le librerie che mi servono:**

In [1]:
import pandas as pd

#### Set pandas options:

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 1000
pd.options.mode.chained_assignment = None

**Importo file CSV**

In [3]:
df = pd.read_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\1_Dataset_Filtrato\\Descrizioni_Filtrate_10.08.2023.csv")

### 1 - Conto quanti beni ho per destinazione d'uso

In [4]:
# Conto quante righe (N° beni) per ogni destinazione d'uso
df_tot_destinazione = df['DestinazioneUso'].value_counts().reset_index()

# Sistemo il nuovo data frame per Destinazione d'uso / Count
df_tot_destinazione = df_tot_destinazione.rename(columns={"index" : "Destinazione_Uso", "DestinazioneUso" : "Count"})

# Plotto il data frame ottenuto
df_tot_destinazione

,Destinazione_Uso,Count
0,RESIDENTIAL,175160
1,STORAGE,21305
2,LAND,11785
3,RETAIL,9576
4,INDUSTRIAL,8972
5,HOTEL,939
6,AGRICULTURAL BUILDING,720


- Posso già notare come la stra grande maggioranza dei dati siano relativi a categorie **RESIDENTIAL**, **STORAGE** & **LAND**.
- Beni con destinazione d'uso inerenti a **RETAIL**, **INDUSTRIAL** sono già sott le 10.000 osservazioni;
- Infine, i beni relativi alla classe **HOTEL** & **AGRICULTURAL BUILDING** sono invece sotto rappresentati all'interno del data frame, con poche centinaia di osservazioni.

### 2 - Analizzo la transcodifica per controllare l'assegnazione delle categorie catastali alle destinazioni d'uso

In [5]:
# Importo ora la transcodifica aziendale. Ad ogni categoria catastale, corrisponde una destinazione d'uso precisa:
df_transcod = pd.read_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\1.1_Dataset_Transcodifiche\\Transcodifiche_Destinazione_Categoria.csv",
                          usecols=['CatCatastale', 'Categoria'])

In [6]:
# Seleziono solo le sei destinazioni d'uso che mi interessa studiare:
df_transcod = df_transcod[(df_transcod["CatCatastale"] == "RESIDENTIAL") | 
                          (df_transcod["CatCatastale"] == "STORAGE") | 
                          (df_transcod["CatCatastale"] == "RETAIL") | 
                          (df_transcod["CatCatastale"] == "AGRICULTURAL BUILDING") | 
                          (df_transcod["CatCatastale"] == "INDUSTRIAL") |
                          (df_transcod["CatCatastale"] == "HOTEL") |
                          (df_transcod["CatCatastale"] == "LAND")]

# Ora raggruppo per ogni classe nella colonna "CatCatastale" ogni unique values contenuta nell'altra colonna "Categoria"
df_transcod = df_transcod.groupby('CatCatastale')['Categoria'].apply(lambda x: x.unique())

# Resetto l'indice e rinomino le colonne del data frame:
df_transcod = df_transcod.to_frame()
df_transcod.reset_index(inplace=True)
df_transcod.rename(columns={"CatCatastale" : "Destinazione_Uso",
                            "Categoria" : "Categoria_Catastale"},
                   inplace=True)

# Mappo una nuova colonna indicando il numero di categorie che compongono ogni singola destinazione d'uso:
df_transcod['N_Categorie'] = df_transcod['Categoria_Catastale'].apply(lambda x: len(x))

# Ordino i valori in ordine alfabeitco:
df_transcod.sort_values(by="N_Categorie",
                        ascending=False,
                        inplace=True)

# Plotto il data frame ottenuto:
df_transcod

,Destinazione_Uso,Categoria_Catastale,N_Categorie
3,RESIDENTIAL,"[A, A1, A2, A3, A4, A5, A7, A8, A9, A10, A11, C6]",12
2,INDUSTRIAL,"[C7, D1, D3, D6, D7, D8, D10]",7
4,RETAIL,"[B, C1, C4]",3
5,STORAGE,"[C2, C3]",2
0,AGRICULTURAL BUILDING,[A6],1
1,HOTEL,[D2],1


- Si evince quindi che la categoria **RESIDENTIAL** comprende il maggior numero di categorie catastali, con ben **12** classi;


- **INDUSTRIAL** invece è al secondo posto con 7 categorie catastali;


- **RETAIL**, **STORAGE**, e **AGRICULTURAL BUILDING**, invece sono caratterizzate da poche categorie catastali (3, 2, 1) di appartenenza.


- Manca solo la clase **LAND** perchè i beni di questa categoria rientrano tuti nel catasto terreno, identificato come categoria catastale **T**;


- Tuttavia, ad un alto numero di categorie catastali, non sempre corrisponde un alto numero di beni associati.

### 3 - Controllo la distribuzione dei beni per categoria catastale su destinazione d'uso con un Pivot

In [7]:
# Creo un Pivot gruppando per categoria catastale e contanto per destinazione d'uso,
# ottenendo il numero di beni totali sia per per categoria catastale sia per destinazione d'uso:
df_tab = pd.crosstab(df["CategoriaCatastale"], df["DestinazioneUso"], margins=True, margins_name='Total')

# Plotto la cross-tab appena creata:
df_tab

DestinazioneUso,AGRICULTURAL BUILDING,HOTEL,INDUSTRIAL,LAND,RESIDENTIAL,RETAIL,STORAGE,Total
CategoriaCatastale,,,,,,,,
A,0,0,0,0,2106,0,0,2106
A1,0,0,0,0,432,0,0,432
A10,0,0,0,0,4085,0,0,4085
A11,0,0,0,0,14,0,0,14
A2,0,0,0,0,50767,0,0,50767
A3,0,0,0,0,38315,0,0,38315
A4,0,0,0,0,18245,0,0,18245
A5,0,0,0,0,1933,0,0,1933
A6,720,0,0,0,0,0,0,720


- **AGRICULTURAL BUILDING** è effettivamente popolata solo dalla categoria **A6**, per un totale di **720** beni;


- **HOTEL** è invece popolata solo in presenza della categoria catastale **D2**, per un totale di **939** beni;


-  **INDUSTRIAL** invece è popolata per un totale di **8.972** beni, appartenenti alle categorie catastali: **C7**, **D1**, **D10**, **D3**, **D6**, **D7** & **D8**;


- **LAND** è popolata da **11.785** beni in totale e sono tutti associati alla categoria catastale **T** & **F1**, che sta per terreno e per parco urbano;
    - Tuttavia, non essendo la categoria catastale **F1** mappata nel file di transcodifica devo eliminare i suddetti beni dal data frame;


- **RESIDENTIAL**, è invece la più popolata, con **175.160** beni associati a questa destinazione d'uso. Al suo interno figurano le categoria catastali:**A1**, **A2**, **A3**, **A4**, **A5**, **A7**, **A8**, **A9**, **A10**, **A11** e **C6**.
    - Tuttavia, figurano anche le categorie catastali **F3**, **F4** & **F5**, le quali non sono presenti nella lista della classe RESIDENTIAL nel file di Trascodifica analizzato prima. Perciò i seguenti beni saranno rimossi dal data frame.


- **RETAIL** è popolata solo dalle categorie catastali **C1** & **C4**, per un totale di **9.576** beni associati;


- **STORAGE** infine è popolata dalle categorie catastali:, **C2** e **C3**, per un totale di **21.305** beni complessivi.


- Fino a qui il check risulta **corretto**, eccetto per le tre categorie catastali (**F2**, **F3**, **F4**) inerenti alla classe residential.

### 4 - Elimino le categorie catastali presenti e non transcodificate

In [8]:
# Conto i beni totali presenti nel data frame:
print("Conto i beni totali presenti nel data frame: ", '\033[1m','\033[15;31;43m', df.shape[0], '\033[0m')

# Specifico le categorie catastali da eliminare:
exc_categories = ['F1', 'F3', 'F4', 'F5']

# Salvo in una variabile tuti i beni da droppare in base alla loro categoria catastale non transcodificata:
drop = ~df['CategoriaCatastale'].isin(exc_categories)

# Elimino i beni che hanno le quattro categorie catastali indicate all'interno del data frame:
df = df[drop]

#Conto i beni totali presenti nel data frame dopo il filtraggio:
print("Conto i beni totali presenti nel data frame dopo il filtraggio: ", '\033[1m','\033[15;31;43m', df.shape[0], '\033[0m')

# Numero beni droppati:
print("Numero beni eliminati: ", '\033[1m','\033[15;31;43m', 228457 - 222736, '\033[0m')

Conto i beni totali presenti nel data frame:    228457 
Conto i beni totali presenti nel data frame dopo il filtraggio:    222736 
Numero beni eliminati:    5721 


### 5 - Secondo Check: Conto il numero di beni associati, filtrandolo per categoria catastale

In [9]:
# 4.1 - Creo per ogni destinazione d'uso una lista con al suo interno tanti items quante
#       sono le categorie catastali ad esse associate.
# 4.2 - Per ogni lista creata, stampo ora il numero complessivo di beni associati alle categorie catastali presenti
#       nelle liste create prima.

RESIDENTIAL = ['A', 'A1', 'A2', 'A3', 'A4', 'A5', 'A7', 'A8', 'A9', 'A10', 'A11', 'C6']
print("Numero di beni appartenenti alla destinazione d'uso RESIDENTIAL: ", 
      '\033[1m','\033[15;31;43m',df['CategoriaCatastale'].isin(RESIDENTIAL).sum(), '\033[0m')

INDUSTRIAL = ['C7', 'D1', 'D3', 'D6', 'D7', 'D8', 'D10']
print("Numero di beni appartenenti alla destinazione d'uso INDUSTRIAL: ", 
      '\033[1m','\033[15;31;43m',df['CategoriaCatastale'].isin(INDUSTRIAL).sum(), '\033[0m')

RETAIL = ['B', 'C1', 'C4']
print("Numero di beni appartenenti alla destinazione d'uso RETAIL: ", 
      '\033[1m','\033[15;31;43m',df['CategoriaCatastale'].isin(RETAIL).sum(), '\033[0m')

STORAGE = ['C2', 'C3']
print("Numero di beni appartenenti alla destinazione d'uso STORAGE: ", 
      '\033[1m','\033[15;31;43m',df['CategoriaCatastale'].isin(STORAGE).sum(), '\033[0m')

AGRICULTURAL_BUILDING = ['A6']
print("Numero di beni appartenenti alla destinazione d'uso AGRICULTURAL BUILDING: ", 
      '\033[1m','\033[15;31;43m',df['CategoriaCatastale'].isin(AGRICULTURAL_BUILDING).sum(), '\033[0m')

HOTEL = ['D2']
print("Numero di beni appartenenti alla destinazione d'uso HOTEL: ", 
      '\033[1m','\033[15;31;43m',df['CategoriaCatastale'].isin(HOTEL).sum(), '\033[0m')

LAND = ['T']
print("Numero di beni appartenenti alla destinazione d'uso LAND: ", 
      '\033[1m','\033[15;31;43m',df['CategoriaCatastale'].isin(LAND).sum(), '\033[0m')

Numero di beni appartenenti alla destinazione d'uso RESIDENTIAL:    170697 
Numero di beni appartenenti alla destinazione d'uso INDUSTRIAL:    8972 
Numero di beni appartenenti alla destinazione d'uso RETAIL:    9576 
Numero di beni appartenenti alla destinazione d'uso STORAGE:    21305 
Numero di beni appartenenti alla destinazione d'uso AGRICULTURAL BUILDING:    720 
Numero di beni appartenenti alla destinazione d'uso HOTEL:    939 
Numero di beni appartenenti alla destinazione d'uso LAND:    10527 


### 6 - Coclusioni

- La **target variable** "Destinazione d'uso" è ora quantitativamente **corretta**;


- La logica espressa nel file di transcodifica aziendale, inizialmente non presentava lo stesso tipo di "Categorie catastali" per ogni "Destinazione d'uso", tuttavia è ora stata corretta eliminando **5.721** beni dal mio data frame;


- Posso ora quindi procedere all'export.

### 7 - Export data frame filtrato e corretto in format CSV

In [10]:
df_csv = df.to_csv("D:\\Master_Cefriel_DS_AI_Monolo\\0_Project_Work\\Dataset\\2_Dataset_Cleaning_1\\Dataset_Cleaning_1.csv",
                   index=False)